## Задание 2

Malicious and Benign Websites

https://www.kaggle.com/xwolf12/malicious-and-benign-websites

Датасет содержит информацию о времени начала и окончания аренды домена (WHOIS), информацию из DNS, закрепленные IP-адреса, используемые версии ПО, размер ответа и кодировку веб-сервиса. Он содержит 21 атрибут, 14 из которых являются количественными. Атрибут Type является меткой, показывающей, является ли данный объект вредоносным сайтом. 

In [1]:
import pandas as pd

df = pd.read_csv('D:\\dataset.csv') # Создадим из датасета объект датафрейма
df.head() # Отобразим первые пять строк

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,...,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263.0,None,None,10/10/2015 18:21,None,...,0,2,700,9,10,1153,832,9,2.0,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087.0,None,None,None,None,...,7,4,1230,17,19,1265,1230,17,0.0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324.0,None,None,None,None,...,0,0,0,0,0,0,0,0,0.0,0
3,B0_113,17,6,ISO-8859-1,nginx,162.0,US,AK,7/10/1997 4:00,12/09/2013 0:45,...,22,3,3812,39,37,18784,4380,39,8.0,0
4,B0_403,17,6,UTF-8,None,124140.0,US,TX,12/05/1996 0:00,11/04/2017 0:00,...,2,5,4278,61,62,129889,4586,61,4.0,0


Видим, что в атрибутах SERVER, CONTENT_LENGTH и DNS_QUERY_TIMES присутствуют пропуски:

In [2]:
df.isnull().sum()

URL                            0
URL_LENGTH                     0
NUMBER_SPECIAL_CHARACTERS      0
CHARSET                        0
SERVER                         1
CONTENT_LENGTH               812
WHOIS_COUNTRY                  0
WHOIS_STATEPRO                 0
WHOIS_REGDATE                  0
WHOIS_UPDATED_DATE             0
TCP_CONVERSATION_EXCHANGE      0
DIST_REMOTE_TCP_PORT           0
REMOTE_IPS                     0
APP_BYTES                      0
SOURCE_APP_PACKETS             0
REMOTE_APP_PACKETS             0
SOURCE_APP_BYTES               0
REMOTE_APP_BYTES               0
APP_PACKETS                    0
DNS_QUERY_TIMES                1
Type                           0
dtype: int64

In [3]:
df = df.dropna() # Исключим все строки с пропусками

In [4]:
quantitative = df.select_dtypes(include=['int64', 'float64']).drop(['Type'], axis=1) # Количественные признаки

for name, data in quantitative.iteritems():
    q_low = data.quantile(0.01) # Значения с наименьшей вероятностью
    q_hi  = data.quantile(0.95) # Значения в доверительном интервале

    quantitative_filtred = quantitative[(quantitative[name] < q_hi) & (quantitative[name] > q_low)] # Исключаем выбросы

Проверим, как выбросы влияют на среднее значение, стандартное отклонение и процентили:

In [5]:
quantitative.describe()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CONTENT_LENGTH,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES
count,967.000000,967.000000,967.000000,967.000000,967.000000,967.000000,967.000000,967.000000,967.000000,9.670000e+02,967.000000,967.000000,967.000000
mean,55.560496,10.808687,11733.697001,16.964840,3.469493,3.188211,1720.409514,19.352637,19.382627,1.702195e+04,1904.967942,19.352637,2.380558
std,25.572456,4.124311,36429.131522,45.139866,6.889008,3.369843,3925.800938,46.072438,50.036140,7.942383e+04,4017.962471,46.072438,2.821823
min,16.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,39.000000,8.000000,324.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,49.000000,10.000000,1853.000000,8.000000,0.000000,2.000000,762.000000,10.000000,10.000000,8.220000e+02,880.000000,10.000000,0.000000
75%,66.500000,12.000000,11324.500000,22.000000,4.000000,5.000000,2332.500000,26.000000,25.000000,9.382000e+03,2765.500000,26.000000,4.000000
max,249.000000,43.000000,649263.000000,1194.000000,58.000000,17.000000,99843.000000,1198.000000,1284.000000,2.060012e+06,100151.000000,1198.000000,20.000000


In [6]:
quantitative_filtred.describe()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CONTENT_LENGTH,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES
count,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,4.110000e+02,411.000000,411.000000,411.000000
mean,54.306569,10.737226,11735.214112,30.773723,3.788321,5.055961,3048.024331,35.026764,35.613139,3.498696e+04,3377.396594,35.026764,4.238443
std,26.916101,4.150704,37806.178733,64.557631,6.273946,3.231594,5414.018316,64.565427,71.250871,1.178971e+05,5421.630258,64.565427,1.448828
min,16.000000,6.000000,0.000000,1.000000,0.000000,1.000000,66.000000,3.000000,4.000000,4.040000e+02,206.000000,3.000000,2.000000
25%,38.000000,8.000000,324.000000,12.000000,0.000000,3.000000,1322.500000,16.000000,15.000000,2.151000e+03,1631.000000,16.000000,4.000000
50%,46.000000,10.000000,1832.000000,20.000000,1.000000,4.000000,2188.000000,24.000000,23.000000,9.200000e+03,2540.000000,24.000000,4.000000
75%,63.000000,12.000000,11003.000000,32.000000,5.000000,7.000000,3420.500000,37.000000,35.000000,2.774150e+04,3772.500000,37.000000,6.000000
max,249.000000,40.000000,435494.000000,1194.000000,58.000000,17.000000,99843.000000,1198.000000,1284.000000,2.060012e+06,100151.000000,1198.000000,6.000000


Проанализируем, как можно преобразовать качественные признаки

In [7]:
df.CHARSET.unique() # Посмотрим, какие значения может принимать признак

array(['iso-8859-1', 'UTF-8', 'us-ascii', 'ISO-8859-1', 'utf-8', 'None',
       'windows-1251', 'ISO-8859'], dtype=object)

In [8]:
df.SERVER.unique()[:10] # Значений достаточно много, при помощи среза [:10] ограничим вывод

array(['nginx', 'Apache/2.4.10', 'Microsoft-HTTPAPI/2.0', 'None',
       'Apache/2', 'Apache/2.2.15 (Red Hat)',
       'Apache/2.4.23 (Unix) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'Apache/2.2.22', 'Apache/2.4.7 (Ubuntu)', 'Apache'], dtype=object)

In [9]:
# Унифицируем значения 
df[['CHARSET','SERVER']] = df[['CHARSET','SERVER']].applymap(lambda x: x.upper()) 

In [10]:
import numpy as np
def server_split(value):
    # Инициализируем переменные, которые будут значениями новых атрибутов
    # В качестве пропусков будем использовать np.NaN чтобы все пропуски были единого типа
    web_server, version, os, openssl = np.NaN, np.NaN, np.NaN, np.NaN 
    values = value.split() # Атрибуты разделены пробелом
    if len(values) > 1: # Если атрибут единственный, в списке будет один элемент
        if '/' in values[0]:
            web_server, version = values[0].split('/')
        else:
            web_server = values[0]
        values.pop(0) # Удаляем элементы из списка по одному, чтобы перебрать оставшиеся и в них найти версию openssl
        if '(' in values[0]:
            os = values[0].strip('(').strip(')')
        values.pop(0)
        for value in values:
            if 'OPENSSL' in value:
                openssl = value.split('/')[1]
                break # Остальные атрибуты в рамках лабораторной извлекать не будем
    return [web_server, version, os, openssl]

In [11]:
new_df_data = list()
for name, data in df['SERVER'].iteritems():
    new_df_data.append(server_split(data))
    
new_df = pd.DataFrame(new_df_data, columns = ['WEB_SERVER','WEB_SERVER_VERSION', 'OS', 'OPENSSL'])
# Добавляем разделенный на части атрибут 
df1 = pd.concat([df, new_df])
df1 = df1.drop('SERVER', axis=1) # Удаляем из датафрейма необработанную колонку

In [12]:
# Преобразуем строковые пропуски 'None' в отсутствующие значения
df1 = df1.replace(to_replace='NONE', value=np.NaN)
df1 = df1.replace(to_replace='None', value=np.NaN)

In [13]:
# Различные реализации веб-сереров лучше преобразовать в составной количественный признак
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
cols_np = enc.fit_transform(df1.WEB_SERVER.to_numpy().reshape(-1, 1)).toarray()
df1 = df1.drop(['WEB_SERVER'], axis=1).join(pd.DataFrame(cols_np, columns=df1.WEB_SERVER.unique()))

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,TCP_CONVERSATION_EXCHANGE,...,ECD,SCRATCH,NGINX,MARRAKESH,ZOPE,AERIA,WWW.LEXISNEXIS.COM,BOSTON.COM,MY,MICROSOFT-IIS
0,M0_109,16.0,7.0,ISO-8859-1,263.0,NaN,NaN,10/10/2015 18:21,NaN,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,B0_2314,16.0,6.0,UTF-8,15087.0,NaN,NaN,NaN,NaN,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,B0_911,16.0,6.0,US-ASCII,324.0,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,B0_62,160.0,29.0,UTF-8,36829.0,US,CA,14/08/1997 0:00,14/06/2016 0:00,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1769,B0_2237,161.0,23.0,ISO-8859-1,417.0,NaN,NaN,23/11/2010 0:00,20/11/2015 0:00,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1774,B0_156,183.0,29.0,ISO-8859-1,4890.0,US,NY,26/06/1997 0:00,18/11/2014 0:00,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1778,B0_162,201.0,34.0,UTF-8,8904.0,US,FL,15/02/1999 0:00,15/07/2015 0:00,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
df1.to_csv('D:\\dataset_clear.csv')